In [1]:
import pandas as pd
import os
import joblib
import shutil
import numpy as np
import cv2

from pathlib import Path
from ipynb.fs.full.preprocessing_image import preprocessing_img
from sklearn.preprocessing import LabelEncoder
from imutils import build_montages

In [2]:
dataset_dir = Path('dataset').resolve()
dicts_temp = {}
df_dataset = pd.DataFrame(columns=['file_path'])

for path, subdirs, files in os.walk(dataset_dir):
    for filename in files:
        if filename.endswith('.png'):
            dicts_temp['file_path'] = Path(os.path.join(path, filename))
            df_temp = pd.DataFrame([dicts_temp])
            df_dataset = pd.concat([df_dataset, df_temp], ignore_index=True)

df_dataset['image_code']    = df_dataset['file_path'].map(lambda x: x.stem)
df_dataset['disease']       = df_dataset['file_path'].map(lambda x: x.parent.stem)
df_dataset['validation']    = df_dataset['file_path'].map(lambda x: x.parent.parent.stem)
df_dataset['geometry_type'] = df_dataset['file_path'].map(lambda x: x.parent.parent.parent.stem)

In [3]:
df_wave_dataset = pd.DataFrame()
df_wave_dataset = df_dataset.loc[(df_dataset['validation'] == 'testing') & (df_dataset['geometry_type'] == 'wave')]
df_wave_dataset = df_wave_dataset.reset_index()
(test_X, test_Y) = preprocessing_img(df_wave_dataset, False)

testing data load:   0%|          | 0/30 [00:00<?, ?it/s]

In [4]:
df_wave_dataset

,index,file_path,image_code,disease,validation,geometry_type
0,102,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V01HO01,healthy,testing,wave
1,103,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V02HO01,healthy,testing,wave
2,104,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V03HO01,healthy,testing,wave
3,105,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V04HO01,healthy,testing,wave
4,106,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V05HO01,healthy,testing,wave
5,107,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V06HO01,healthy,testing,wave
6,108,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V07HO01,healthy,testing,wave
7,109,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V08HO01,healthy,testing,wave
8,110,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V09HO01,healthy,testing,wave
9,111,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V10HO01,healthy,testing,wave


In [5]:
model_joblib = joblib.load('./model/finalized_wave_model.joblib')

# encode the labels as integers
le = LabelEncoder()
train_Y = test_Y
train_Y = le.fit_transform(train_Y)
test_Y = le.transform(test_Y)

pred_Y = model_joblib.predict(test_X)
label = le.inverse_transform(pred_Y)
df_wave_dataset = pd.concat([df_wave_dataset, pd.DataFrame(label, columns = ['pred_Y'])], axis=1)

In [6]:
df_wave_dataset

,index,file_path,image_code,disease,validation,geometry_type,pred_Y
0,102,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V01HO01,healthy,testing,wave,healthy
1,103,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V02HO01,healthy,testing,wave,healthy
2,104,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V03HO01,healthy,testing,wave,parkinson
3,105,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V04HO01,healthy,testing,wave,healthy
4,106,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V05HO01,healthy,testing,wave,parkinson
5,107,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V06HO01,healthy,testing,wave,parkinson
6,108,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V07HO01,healthy,testing,wave,healthy
7,109,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V08HO01,healthy,testing,wave,parkinson
8,110,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V09HO01,healthy,testing,wave,healthy
9,111,C:\Users\ZHENGYU\Desktop\P2\dataset\wave\testi...,V10HO01,healthy,testing,wave,healthy


In [7]:
Path("./result/parkinson").mkdir(parents = True, exist_ok = True)
Path("./result/healthy").mkdir(parents = True, exist_ok = True)
Path("./output").mkdir(parents = True, exist_ok = True)

In [8]:
for index, row in df_wave_dataset.iterrows():
    source = row['file_path']
    destination = Path('result/' + row['pred_Y']).resolve()/source.name
    shutil.copyfile(str(source), str(destination))

In [9]:
# randomly select a few images and then initialize the output images
# for the montage
idxs = np.arange(0, len(df_wave_dataset))
idxs = np.random.choice(idxs, size=(25,), replace=False)
selected_images = []

In [10]:
for i in idxs:
    # load the testing image, clone it, and resize it
    img_input = cv2.imread(str(df_wave_dataset['file_path'][i]))
    img_output = img_input.copy()
    img_output = cv2.resize(img_output, (128, 128))
    
    # draw the colored class label on the output image and add it to
    # the set of output images
    label = df_wave_dataset['pred_Y'][i]
    color = (0, 255, 0) if label == "healthy" else (0, 0, 255)
    cv2.putText(img_output, label, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
        color, 2)
    selected_images.append(img_output)

In [11]:
# create a montage using 128x128 "tiles" with 5 rows and 5 columns
montage = build_montages(selected_images, (128, 128), (5, 5))[0]

In [12]:
# Filename
filename = './output/wave_prediction_result.jpg'
cv2.imwrite(filename, montage)

True

In [13]:
# show the output montage
cv2.imshow("Wave Prediction Result", montage)
cv2.waitKey(0)
cv2.destroyAllWindows()